# Détection automatique d'erreurs d'OCR

## Installation

In [1]:
!pip install thefuzz[speedup]

     |████████████████████████████████| 50 kB 1.0 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp39-cp39-linux_x86_64.whl size=178788 sha256=df16631ecc0f3a6f2b1d9980bebc3f8d4b0cb5ad3c88f476746f79bc2afb77df
  Stored in directory: /root/.cache/pip/wheels/46/4a/6c/164a1d9dd67c82d208f19d869ad0a517a0c5a6117f608c53e6
Successfully built python-levenshtein
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [10]:
!pip install --upgrade pip

     |████████████████████████████████| 1.7 MB 1.3 MB/s            
  Attempting uninstall: pip
    Found existing installation: pip 21.3
    Uninstalling pip-21.3:
      Successfully uninstalled pip-21.3


## Imports

In [2]:
import nltk
from thefuzz import fuzz
from thefuzz import process
from Levenshtein import distance

Nous allons d'abord utiliser la distance de Levenshtein, selon sa formule originale (1965)

In [3]:
distance("bulletin", "ballotins")

3

Nous allons ensuite faire appel à la librairie TheFuzz (anciennement fuzzywuzzy) qui repose sur python-Levenshtein (https://github.com/seatgeek/thefuzz).
Celle-ci permet de normaliser le résultat en calculant un ratio (pourcentage) de similarité entre deux chaînes de caractères :

In [4]:
fuzz.ratio("bulletin", "ballotins")

71

La fonction  `partial_ratio` se focalise sur la sous-chaîne (substring) la plus similaire : 

In [5]:
fuzz.partial_ratio("Grand", "Grand-Place")

100

La fonction `token_sort_ratio` ignore l'ordre des mots :

In [6]:
fuzz.token_sort_ratio("archives de la ville de Bruxelles", "de Bruxelles la ville de archives")

100

La fonction `token_set_ratio` ignore les répétitions de mots :

In [7]:
fuzz.token_set_ratio("plus jamais ça !", "plus jamais Jamais JAMAIS ça !!!")

100

Chargeons à présent le fichier `all.txt` (tronqué) : 

In [8]:
data_path = "../data/all.txt"
LIMIT = 50000000
text = open(data_path).read()[:LIMIT]

tokens = nltk.wordpunct_tokenize(text)
print(f"{len(tokens)} words (tokens) found")

vocab = set(tokens)
print(f"{len(vocab)} different word forms")

12683666 words (tokens) found
159024 different word forms


Nous pouvons identifier les variantes d'un mot par ordre décroissant de similarité : 

In [9]:
word = "bruxelles"
process.extract(word, vocab, limit=15)

[('âBruxelles', 100),
 ('BRUXELLES', 100),
 ('bruxelles', 100),
 ('Bruxelles', 100),
 ('àBruxelles', 100),
 ('Bruxellles', 95),
 ('BRUXELLEST', 95),
 ('àBruxellesj', 95),
 ('ruxelles', 94),
 ('BRÜXELLES', 94),
 ('Bruxelle', 94),
 ('Bruxelleà', 94),
 ('Bruxeles', 94),
 ('Bruxlles', 94),
 ('Elle', 90)]

Les caractères accentués ne sont pas pris en compte pour le calcul du score, mais cela reste une bonne base pour récupérer les occurrences mal OCRisées d'un mot.